In [ ]:
import threading
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import numpy as np
import time
import cvlib as cv
from cvlib.object_detection import draw_bbox
import math
import os
import glob

#tkinter window initialized
root = Tk()
root.title("Security Camera")
root.geometry("1000x600")

#frames for video and controls initialized
video_frame = Frame(root, bg="black", width=800, height=600)
video_frame.grid(row=0, column=0, sticky="nsew")
control_frame = Frame(root, bg="gray", width=200, height=600)
control_frame.grid(row=0, column=1, sticky="nsew")

root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=4)
root.grid_columnconfigure(1, weight=1)

lmain = Label(video_frame)
lmain.pack(expand=True)

# Global variables
cap = None
pause = False
current_frame = None
previous_frame = None
previous_time = time.time()
video_writer = None
is_recording = False
tracked_objects = {}
object_id_counter = 1
SOME_THRESHOLD = 40 
FIXED_DISTANCE_CAR = 4.2 
FIXED_DISTANCE_TRUCK = 6.0  
FIXED_DISTANCE_BUS = 8.0 
SPEED_BUFFER_DURATION = 1

# Function to calculate Euclidean distance
def calculate_distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

def calculate_speed_with_fixed_distance(time_elapsed, fixed_distance):
    if time_elapsed > 0:
        speed_m_per_sec = fixed_distance / time_elapsed
        return speed_m_per_sec * 3.6
    return 0


def detect_objects(frame):
    bbox, label, confidences = cv.detect_common_objects(frame, confidence=0.5, model='yolov3')
    return bbox, label, confidences

def start_recording():
    global video_writer, is_recording, current_frame
    if current_frame is not None and not is_recording:
        is_recording = True
        filename = "recorded_video.mp4"
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
        video_writer = cv2.VideoWriter(filename, fourcc, 20.0, (current_frame.shape[1], current_frame.shape[0]))
        btn_start_recording.config(text="Recording Started...", bg="red") 
        print("Recording started")

def stop_recording():
    global video_writer, is_recording
    if is_recording:
        is_recording = False
        video_writer.release()
        btn_start_recording.config(text="Start Recording", bg="SystemButtonFace")  and color
        print("Recording stopped")
        
def apply_noise_distortion(frame, noise_level=0.05):
    noisy_frame = np.copy(frame).astype(float)
    noise = np.random.randn(*frame.shape) * noise_level
    noisy_frame += noise * 255
    np.clip(noisy_frame, 0, 255, out=noisy_frame)
    return noisy_frame.astype(np.uint8)


def play_video():
    global pause, current_frame, previous_frame, video_writer, is_recording, previous_time, tracked_objects, object_id_counter
    if cap is None or not cap.isOpened():
        return
    if pause:
        lmain.after(10, play_video)
        return
    ret, frame = cap.read()
    if not ret:
        return
    
    frame = apply_noise_distortion(frame)
    current_frame = frame
    if is_recording:
        video_writer.write(current_frame)

    bbox, label, confidences = detect_objects(frame)
    out = draw_bbox(frame, bbox, label, confidences)

    current_time = time.time()
    time_elapsed = current_time - previous_time

    new_tracked_objects = {}
    for box, obj_label in zip(bbox, label):
        if obj_label in ['car', 'truck', 'bus']:
            fixed_distance = FIXED_DISTANCE_CAR
            if obj_label == 'truck':
                fixed_distance = FIXED_DISTANCE_TRUCK
            elif obj_label == 'bus':
                fixed_distance = FIXED_DISTANCE_BUS

            matched_id, min_distance = None, float('inf')
            for obj_id, obj_info in tracked_objects.items():
                prev_box = obj_info['bbox']
                distance = calculate_distance((box[0], box[1]), (prev_box[0], prev_box[1]))
                if distance < min_distance:
                    matched_id, min_distance = obj_id, distance

            if matched_id is None or min_distance > SOME_THRESHOLD:
                matched_id = object_id_counter
                object_id_counter += 1
                new_tracked_objects[matched_id] = {'bbox': box, 'last_updated': current_time, 'speed': 0}
            else:
                prev_info = tracked_objects[matched_id]
                time_since_last_update = current_time - prev_info['last_updated']
                if time_since_last_update <= SPEED_BUFFER_DURATION:
                    actual_distance = calculate_distance((box[0], box[1]), (prev_info['bbox'][0], prev_info['bbox'][1]))
                    new_speed = calculate_speed_with_fixed_distance(time_elapsed, fixed_distance)
                    if new_speed > 0:
                        speed = new_speed
                    else:
                        speed = prev_info['speed']  
                else:
                    speed = 0

                new_tracked_objects[matched_id] = {'bbox': box, 'last_updated': current_time, 'speed': speed}

            speed = new_tracked_objects[matched_id]['speed']
            cv2.putText(out, f"Speed: {speed:.2f} km/h", (box[0], box[3] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    tracked_objects = new_tracked_objects
    previous_frame = current_frame
    previous_time = current_time

    cv2image = cv2.cvtColor(out, cv2.COLOR_BGR2RGBA)
    img = Image.fromarray(cv2image)
    img = img.resize((640, 480))
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(10, play_video)
    
def release_and_open_cap(source):
    global cap
    if cap is not None:
        cap.release()
    cap = cv2.VideoCapture(source)

def select_video():
    global pause
    video_path = filedialog.askopenfilename()
    if video_path:
        pause = False
        release_and_open_cap(video_path)
        play_video()

def initialize_camera():
    global cap, pause
    pause = False
    cap = cv2.VideoCapture(0)  
    if not cap.isOpened():
        print("Failed to access the webcam.")
    else:
        print("Webcam accessed successfully.")
        play_video()

#'go_live' function to just call 'initialize_camera'
def go_live():
    initialize_camera()
    
def pause_video():
    global pause
    pause = not pause

def stop_video():
    global cap
    if cap is not None:
        cap.release()
        cap = None
    lmain.configure(image='')

capture_count = 0
captured_frames = []

def capture_frame():
    global capture_count, current_frame, captured_frames
    if current_frame is not None:
        capture_count += 1
        filename = f"captured_frame_{capture_count}.jpg"
        cv2.imwrite(filename, current_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 80])  # Save frame
        print(f"Frame captured as '{filename}'")
        captured_frames.append(current_frame) 

def stitch_frames():
    global captured_frames
    if len(captured_frames) < 2:
        print("Need at least two frames to stitch.")
        return

    # stitcher ceated and used to stitch frames
    stitcher = cv2.Stitcher_create()
    status, stitched = stitcher.stitch(captured_frames)

    if status == cv2.Stitcher_OK:
        # Save the stitched image
        filename = f"stitched_image_{time.strftime('%Y%m%d-%H%M%S')}.jpg"  # Using timestamp
        cv2.imwrite(filename, stitched)
        print(f"Stitched image saved as '{filename}'")
        
        #show the stitched image
        cv2.imshow('Stitched Image', stitched)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Stitching failed. Error code: {}".format(status))

    captured_frames = [] 

def on_closing():
    cv2.destroyAllWindows()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)


button_height = 2
button_width = 20
btn_select = Button(control_frame, text="Select Video", command=select_video, height=button_height, width=button_width)
btn_play = Button(control_frame, text="Play", command=play_video, height=button_height, width=button_width)
btn_pause = Button(control_frame, text="Pause/Resume", command=pause_video, height=button_height, width=button_width)
btn_stop = Button(control_frame, text="Stop", command=stop_video, height=button_height, width=button_width)
btn_capture = Button(control_frame, text="Capture Frame", command=capture_frame, height=button_height, width=button_width)
btn_go_live = Button(video_frame, text="Go Live", command=go_live, height=button_height, width=button_width)
btn_start_recording = Button(video_frame, text="Start Recording", command=start_recording, height=button_height, width=button_width)
btn_stop_recording = Button(video_frame, text="Stop Recording", command=stop_recording, height=button_height, width=button_width)
btn_stitch = Button(video_frame, text="Stitch Frames", command=stitch_frames, height=button_height, width=button_width)

btn_go_live.pack(side=LEFT, padx=20)
btn_start_recording.pack(side=LEFT, padx=20)
btn_stop_recording.pack(side=LEFT, padx=20)

btn_stitch.pack(side=LEFT, padx=20)

buttons = [btn_select, btn_play, btn_pause, btn_stop, btn_capture]
for i, btn in enumerate(buttons):
    btn.place(relx=0.5, rely=(0.1 + i * 0.15), anchor=CENTER)



# Call initialize_camera at the end of the script to start the live feed immediately
initialize_camera()
# Start the tkinter loop
root.mainloop()


Webcam accessed successfully.
